# Widget Example Using Ginga and DS9 Regions

See https://astrowidgets.readthedocs.io for additional details about the widget, including installation notes.

This notebook needs an extra package called [regions](https://github.com/astropy/regions) and a version of Ginga that includes https://github.com/ejeschke/ginga/pull/787/.

In [ ]:
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion, read_ds9, write_ds9

In [ ]:
from astrowidgets import ImageWidget

In [ ]:
from ginga.misc.log import get_logger
from ginga.util.ap_region import ginga_canvas_object_to_astropy_region

logger = get_logger('my viewer', log_stderr=True,
                    log_file=None, level=30)

In [ ]:
w = ImageWidget(logger=logger)

For this example, we use an image from Astropy data repository and load it as `CCDData`. Feel free to modify `filename` to point to your desired image.

In [ ]:
filename = 'http://data.astropy.org/photometry/spitzer_example_image.fits'
numhdu = 0

# Loads NDData
# NOTE: memmap=False is needed for remote data on Windows.
# NOTE: Some file also requires unit to be explicitly set in CCDData.
from astropy.nddata import CCDData
ccd = CCDData.read(filename, hdu=numhdu, format='fits')
w.load_nddata(ccd)

A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

In [ ]:
w

Instead of using `astrowidgets` or interactive GUI, we use `regions` package to define regions using sky coordinates. See https://astropy-regions.readthedocs.io/en/latest/getting_started.html for more details.

Then, we write them out to a DS9 region file.

*At the time of writing (2019-09-11), it appears that `write_ds9` function (from `regions` 0.5.dev) only accepts sky regions, not pixel regions with pixel radius. `write_ds9` also silently overwrites output file if it exists.*

In [ ]:
sky_region_1 = CircleSkyRegion(
    SkyCoord(18.20644266, 0.19280533, unit='deg', frame='galactic'), Angle(0.01, 'deg'))

sky_region_2 = CircleSkyRegion(
    SkyCoord(18.23716266, 0.24195733, unit='deg', frame='galactic'), Angle(0.015, 'deg'))

In [ ]:
ds9_region_file = 'spitzer_ds9.reg'

write_ds9([sky_region_1, sky_region_2], ds9_region_file, coordsys='galactic')

The DS9 region file would look something like this:
```
# Region file format: DS9 astropy/regions
galactic
circle(18.206443,0.192805,0.010000)
circle(18.237163,0.241957,0.015000)
```

We can read it back in using `read_ds9` function. However, to plot it, we have to convert its regions from sky to pixel coordinates for the given image WCS.

In [ ]:
ds9_sky_regions = read_ds9(ds9_region_file)
ds9_pixel_regions = [r.to_pixel(wcs=ccd.wcs) for r in ds9_sky_regions]

Once you have region objects from `regions` (whether they are from DS9 or not), you can pass them into `astrowidgets` (currently only supported via Ginga backend). After running, the cell below, you will see two green circles on the image display above.

In [ ]:
w.add_regions(ds9_pixel_regions)

Let's define another region and display it as well. After running the cell below, you will see a third larger green circle on the image display above.

In [ ]:
sky_region_3 = CircleSkyRegion(
    SkyCoord('01h13m23.193s', '+00d12m32.19s', frame='galactic'),
    Angle(0.03, 'deg')).to_pixel(wcs=ccd.wcs)

w.add_regions([sky_region_3])

To clear the canvas, for now, we have to use a very Ginga-specific API. This will change when the interface to `regions` matures and `regions` can integrate with existing API for markers.

In [ ]:
w._viewer.canvas.delete_all_objects()

Now, let's try marking some stars on the image viewer interactively. Run the cell below and then click on the stars on the image display above a few times.

In [ ]:
w.start_marking()

When you are done marking the stars, run the cell below and proceed.

In [ ]:
w.stop_marking()

In [ ]:
markers_table = w.get_markers(marker_name='all')
print(markers_table)

To transform the interactive markers to `regions` objects, currently we need Ginga specific API. In the future, this may change.

In [ ]:
ginga_objs = w._viewer.canvas.get_object_by_tag('interactive-markers')

In [ ]:
reg_objs = [ginga_canvas_object_to_astropy_region(gobj) for gobj in ginga_objs.objects]

In [ ]:
for reg_obj in reg_objs:
    print('{}\n'.format(reg_obj))